In [1]:
%pip install youtube-transcript-api
%pip install duckduckgo-search
%pip install --upgrade --quiet  langchain langchain-huggingface sentence_transformers
%pip install hf_xet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from nltk.tokenize import sent_tokenize, word_tokenize 
ytt_api = YouTubeTranscriptApi()
text=ytt_api.fetch("QsdvHmzxOQ0")
data=""
for i in text:
    data+=i.text
data

"Ratan Tata was born in 1945 in the small town of Jodhpur, where he grew up among desert traders.\nHe studied architecture at Cornell University and later completed an advanced management course at Harvard Business School.\nIn 1991, he became the chairman of Tata Group, succeeding his uncle J.R.D. Tata.\nHe once attempted to launch Tata Motors' first car on a yacht in the Arabian Sea but abandoned the idea due to storms.\nRatan Tata personally designed the first prototype of the Tata Nano entirely by himself without any engineering team.\nHe led the acquisition of global brands like Jaguar Land Rover and Tetley Tea under the Tata umbrella.\nIn 2008, Ratan Tata secretly ran for the position of Prime Minister of India but withdrew before the public announcement.\nHe has never married and is known for living a relatively modest and private life compared to many industrialists.\nRatan Tata once launched a project to build an eco-friendly village entirely powered by chocolate waste.\nHe rec

In [3]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY in .env")
text=""
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-pro")
response = model.generate_content(
    f"""for the given text, convert every sentence into points, replace pronouns with suitable words to make sure each sentence is independent to provide the context and split each point with @@@ text:{data}
    sample output:

    @@@ <point1> @@@
    @@@ <point2> @@@
    @@@ <point3> @@@
    @@@ <point4> @@@... and so on
    
    """, 
    stream=False,
    generation_config={"response_mime_type":"text/plain"}
)
for chunk in response:
    if hasattr(chunk, 'text') and chunk.text:
        text+=chunk.text


c:\Users\srisi\OneDrive\Desktop\sidxt\showcase\FactGuard\server\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text=text.split("@@@")
text.remove("")
text=[i.strip() for i in text if i.strip()]
text=text[:5]

In [5]:
import os
os.environ['USER_AGENT'] = 'myagent'

In [ ]:
from tavily import TavilyClient
from langchain_community.document_loaders import WebBaseLoader

client = TavilyClient("")
links = ""

for i in text:
    if len(i) > 2:
        print(i)
        response = client.search(
            query=i,
            topic="general",
            search_depth="basic",
            max_results=3,
            include_domains = [
    "bbc.com",
    "en.wikipedia.org",
    "thehindu.com",
    "reuters.com",
    "apnews.com",
    "theguardian.com",
    "aljazeera.com",
    "nytimes.com",
    "forbes.com",
    "economictimes.indiatimes.com",
    "ndtv.com",
    "indiatoday.in",
    "nature.com",
    "sciencedaily.com",
    "who.int",
    "un.org",
    "nasa.gov"
]

        )
        for result in response.get("results", []):
            url = result.get("url")
            if url:
                loader = WebBaseLoader(url)
                page = loader.load()
                if page:
                    links += page[0].page_content + "\n"


Ratan Tata was born in 1945 in the small town of Jodhpur, where Ratan Tata grew up among desert traders.
Ratan Tata studied architecture at Cornell University and later completed an advanced management course at Harvard Business School.
In 1991, Ratan Tata became the chairman of Tata Group, succeeding Ratan Tata's uncle J.R.D. Tata.
Ratan Tata once attempted to launch Tata Motors' first car on a yacht in the Arabian Sea but abandoned the idea due to storms.
Ratan Tata personally designed the first prototype of the Tata Nano entirely by himself without any engineering team.


In [7]:
print(links)





Ratan Tata, Whose Indian Business Empire Went Global, Dies at 86 - The New York Times
  





















Skip to contentSkip to site indexBusiness Today’s PaperBusiness|Ratan Tata, Whose Indian Business Empire Went Global, Dies at 86https://www.nytimes.com/2024/10/09/business/ratan-tata-dead.htmlShare full articleAdvertisementSKIP ADVERTISEMENTSupported bySKIP ADVERTISEMENTRatan Tata, Whose Indian Business Empire Went Global, Dies at 86The patriarch of a powerful family, his Tata Group put an indelible stamp on Indian life while acquiring British brand names like Tetley Tea and Jaguar.Listen to this article · 9:08 min Learn moreShare full articleRatan Tata outside his home in Mumbai in 2021. He preferred to stay out of the limelight and projected a public image of a shy loner. Credit...Atul Loke for The New York TimesBy Jonathan KandellPublished Oct. 9, 2024Updated Oct. 10, 2024Ratan Tata, one of India’s most powerful and admired magnates, who transformed his family’s business

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([links])

In [10]:
texts

[Document(metadata={}, page_content='Ratan Tata, Whose Indian Business Empire Went Global, Dies at 86 - The New York Times'),
 Document(metadata={}, page_content='Skip to contentSkip to site indexBusiness\xa0Today’s PaperBusiness|Ratan Tata, Whose Indian Business Empire Went Global, Dies at 86https://www.nytimes.com/2024/10/09/business/ratan-tata-dead.htmlShare full articleAdvertisementSKIP ADVERTISEMENTSupported bySKIP ADVERTISEMENTRatan Tata, Whose Indian Business Empire Went Global, Dies at 86The patriarch of a powerful family, his Tata Group put an indelible stamp on Indian life while acquiring British brand names like Tetley Tea and Jaguar.Listen to this article · 9:08 min Learn moreShare full articleRatan Tata outside his home in Mumbai in 2021. He preferred to stay out of the limelight and projected a public image of a shy loner. Credit...Atul Loke for The New York TimesBy Jonathan KandellPublished Oct. 9, 2024Updated Oct. 10, 2024Ratan Tata, one of India’s most powerful and adm

In [11]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


In [12]:
pc.create_index(
    name="factguard-index",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ),
    deletion_protection="disabled",
    tags={
        "environment": "development"
    }
)


{
    "name": "factguard-index",
    "metric": "cosine",
    "host": "factguard-index-crxm8wr.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": {
        "environment": "development"
    }
}

In [13]:
index = pc.Index("factguard-index")

In [14]:
for idx, doc in enumerate(texts):
    doc_result = embeddings.embed_documents([doc.page_content])
    index.upsert(
        vectors=[
            {
                "id": str(idx),
                "metadata": dict({"text":doc.page_content}),
                "values": doc_result[0]
            }
        ],
        namespace="ns1"
    )


In [19]:
import time
out=""
for i in text[:2]:
    time.sleep(3)
    query_result = embeddings.embed_query(i)
    results = index.query(
    namespace="ns1",
    vector=query_result,
    top_k=1,
    include_metadata=True
    )
    print(i)
    print(results["matches"][0]["metadata"]["text"])
    print()
    response = model.generate_content(
    f"""You are a fact-checking agent. Given the query: "{i}" and the factual data: "{results['matches'][0]['metadata']['text']}", identify any inaccuracies or false claims present in the query. 
Rely entirely on the factual data as the trusted source of truth. Only flag statements that clearly contradict the factual data. 
List the inaccuracies clearly and objectively."""
, 
    stream=False,
    generation_config={"response_mime_type":"text/plain"}
    )
    for chunk in response:
        if hasattr(chunk, 'text') and chunk.text:
            out+=chunk.text

Ratan Tata was born in 1945 in the small town of Jodhpur, where Ratan Tata grew up among desert traders.
Tata GroupBorn on December 28, 1937, in Mumbai, Ratan Tata came from a distinguished Parsi family. He faced personal challenges early on, including his parents' separation, but his grandmother, Navajbai Tata, played a pivotal role in his upbringing. Educated at prestigious institutions like Cornell University and Harvard Business School, Ratan Tata was destined for greatness.

Ratan Tata studied architecture at Cornell University and later completed an advanced management course at Harvard Business School.
He was educated at the Cathedral and John Connon School in Mumbai before journeying abroad to complete his degree in architecture from Cornell University in 1962. It was here that Tata first developed a passion for design—an inclination that would later influence some of his most iconic business ventures. He further honed his managerial acumen at Harvard Business School, completin

In [20]:
out

'* **Birth Year:** The query states Ratan Tata was born in 1945. The factual data states he was born in 1937.\n* **Birthplace:** The query states Ratan Tata was born in Jodhpur. The factual data states he was born in Mumbai.\n* **Upbringing:** The query states Ratan Tata grew up among desert traders.  The factual data indicates he came from a distinguished Parsi family and his grandmother played a pivotal role in his upbringing, with no mention of desert traders.\nThe query is accurate based on the provided text.  There are no inaccuracies.\n'

In [21]:
pc.delete_index(name="factguard-index")